In [1]:
import pandas as pd
import shutil, os
import numpy as np

In [2]:
df = pd.read_csv('p.csv')
df.head()

,parcel_name,sws_rs,sws_gb,sws_ms,sws_fa,sws_rw,sws_total
0,gmd_17Networks_17,0.071966,0.582126,0.248658,0.455413,0.046039,0.231128
1,gmd_17Networks_16,0.201671,0.703188,0.776469,0.457278,0.285506,0.323083
2,gmd_17Networks_15,0.665409,0.773109,0.810119,0.239218,0.956343,0.521962
3,gmd_17Networks_14,0.635989,0.410975,0.438266,0.596597,0.562483,0.666003
4,gmd_17Networks_13,0.212942,0.968948,0.120675,0.291418,0.113960,0.360521


In [3]:
df1 = df[['parcel_name' ,'sws_gb']]
df1.head()

,parcel_name,sws_gb
0,gmd_17Networks_17,0.582126
1,gmd_17Networks_16,0.703188
2,gmd_17Networks_15,0.773109
3,gmd_17Networks_14,0.410975
4,gmd_17Networks_13,0.968948


In [4]:
df1.shape

(2718, 2)

In [5]:
df2 = df1.dropna()

In [6]:
df2.shape

(2718, 2)

In [7]:
df2.insert(1, 'parc_name', np.nan)

In [8]:
df2.head()

,parcel_name,parc_name,sws_gb
0,gmd_17Networks_17,NaN,0.582126
1,gmd_17Networks_16,NaN,0.703188
2,gmd_17Networks_15,NaN,0.773109
3,gmd_17Networks_14,NaN,0.410975
4,gmd_17Networks_13,NaN,0.968948


In [9]:
df2 = df2.reset_index()

In [10]:
df2.head()

,index,parcel_name,parc_name,sws_rs
0,0,gmd_17Networks_17,NaN,0.071966
1,1,gmd_17Networks_16,NaN,0.201671
2,2,gmd_17Networks_15,NaN,0.665409
3,3,gmd_17Networks_14,NaN,0.635989
4,4,gmd_17Networks_13,NaN,0.212942


In [11]:
df = df2.drop('index', axis = 1)

In [9]:
df = df2

In [10]:
df.head()

,parcel_name,parc_name,sws_gb
0,gmd_17Networks_17,NaN,0.582126
1,gmd_17Networks_16,NaN,0.703188
2,gmd_17Networks_15,NaN,0.773109
3,gmd_17Networks_14,NaN,0.410975
4,gmd_17Networks_13,NaN,0.968948


In [11]:
markers = ['gmd_', 'zALFF_', 'zfALFF_', 'zReHo_', 'zDCwei_', 'zDCbin_']
for i,parc in enumerate(df["parcel_name"]):
    #for i in range(df.shape[0]):
    for marker in markers:
        if parc.find(marker)==0:
            df["parc_name"][i] = df["parcel_name"][i][len(marker):]
            #print(df["sws_total"][i])
            col = marker + "sws_gb_p" 
            if col not in df:
                df.insert(2, col, np.nan)
            df.iloc[i, df.columns.get_loc(col)] = df["sws_gb"][i]

<ipython-input-11-ba0fa4ffe3e6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["parc_name"][i] = df["parcel_name"][i][len(marker):]
/home/tara/Programming/env/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
df.shape

(2718, 9)

In [15]:
#df.to_csv('p_reformed.csv')

### merging duplicates

In [16]:
#df = pd.read_csv('p_reformed.csv')
#df.head()

In [13]:
df.shape

(2718, 9)

In [14]:
df2 = df.copy()
inds = df2['parc_name'].drop_duplicates().reset_index(drop=True)
df2.set_index('parc_name', inplace=True)
df3 = pd.DataFrame(columns = df2.columns)
for ind in inds:
    region = df2.loc[[ind]]
    if region.shape[0] > 1 :
        reg = region
        cols = reg.columns
        for col in cols:
            reg[col] = reg[col].sum()
        reg = reg.drop_duplicates()
        df3 = pd.concat([df3, reg])
        continue
    df3 = pd.concat([df3, region])

In [16]:
df3.to_csv('SWSgb_p_reformed.csv')

In [15]:
df3.shape

(453, 8)

In [17]:
df3.head()

,parcel_name,zALFF_sws_gb_p,zDCbin_sws_gb_p,zDCwei_sws_gb_p,zfALFF_sws_gb_p,zReHo_sws_gb_p,gmd_sws_gb_p,sws_gb
17Networks_17,gmd_17Networks_17zReHo_17Networks_17zfALFF_17N...,0.366094,0.819455,0.764998,0.949608,0.419655,0.582126,3.901935
17Networks_16,gmd_17Networks_16zReHo_17Networks_16zfALFF_17N...,0.544756,0.639458,0.680816,0.695803,0.979621,0.703188,4.243643
17Networks_15,gmd_17Networks_15zReHo_17Networks_15zfALFF_17N...,0.492052,0.427861,0.509473,0.250155,0.478610,0.773109,2.931260
17Networks_14,gmd_17Networks_14zReHo_17Networks_14zfALFF_17N...,0.535261,0.879664,0.889848,0.941452,0.920433,0.410975,4.577634
17Networks_13,gmd_17Networks_13zReHo_17Networks_13zfALFF_17N...,0.464271,0.797318,0.753311,0.542369,0.924851,0.968948,4.451068


In [22]:
data = pd.read_csv('SWSrs_p_reformed.csv')
data.head()

,Unnamed: 0,parcel_name,zALFF_sws_rs_p,zDCbin_sws_rs_p,zDCwei_sws_rs_p,zfALFF_sws_rs_p,zReHo_sws_rs_p,gmd_sws_rs_p,sws_rs
0,17Networks_17,gmd_17Networks_17zReHo_17Networks_17zfALFF_17N...,0.259976,0.588196,0.563896,0.888731,0.300647,0.071966,2.673412
1,17Networks_16,gmd_17Networks_16zReHo_17Networks_16zfALFF_17N...,0.620199,0.397304,0.346826,0.282391,0.905764,0.201671,2.754155
2,17Networks_15,gmd_17Networks_15zReHo_17Networks_15zfALFF_17N...,0.626217,0.075195,0.055932,0.075517,0.004952,0.665409,1.503222
3,17Networks_14,gmd_17Networks_14zReHo_17Networks_14zfALFF_17N...,0.974232,0.291427,0.213209,0.479685,0.569029,0.635989,3.163571
4,17Networks_13,gmd_17Networks_13zReHo_17Networks_13zfALFF_17N...,0.966108,0.595933,0.617467,0.424920,0.559959,0.212942,3.377329
